# Google Analytics Customer Revenue Prediction

In [46]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import os
import matplotlib.pyplot as plt


In [13]:
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(
        csv_path,
        converters={ column: json.loads for column in JSON_COLUMNS },
        dtype={ 'fullVisitorId': 'str', 'visitId': 'str' },
        nrows=nrows
    )
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f'Loaded {os.path.basename(csv_path)}. Shape: {df.shape}')
    return df

In [15]:
%%time
df_train = load_df()
df_test = load_df('../input/test.csv')

%%time
df_train.to_csv('train-flattened.csv', index=False)
df_test.to_csv('test-flattened.csv', index=False)

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
CPU times: user 4min 44s, sys: 8.12 s, total: 4min 52s
Wall time: 4min 52s


In [20]:
df_train.shape, df_test.shape

((903653, 55), (804684, 53))

In [53]:
df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)

11515